In [30]:
import tensorflow as tf
from tensorflow.python.framework import ops
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

In [3]:
from numpy.random import rand, choice
npop = 100
nsets = 10
Cmat = (rand(nsets,npop) > rand(nsets, 1)) + 0.0

In [4]:
csizes = np.sum(Cmat, 1)

In [24]:
nsub = 30
csizes_scale = (csizes * nsub/npop)
csizes_scale = csizes_scale.reshape((-1, 1))
csizes_scale.shape

(10, 1)

In [113]:
nrestarts = 20
ops.reset_default_graph()
lam_bin = tf.placeholder(dtype = tf.float32, shape = ())
lam_count = tf.placeholder(dtype = tf.float32, shape = ())
lam_loss = tf.placeholder(dtype = tf.float32, shape = ())
x = tf.get_variable(dtype = tf.float32, shape = (npop, nrestarts), name = 'x',
                     initializer = tf.random_normal_initializer(mean = 0.5, stddev = 0.1))
C = tf.constant(Cmat, dtype = tf.float32)
cs = tf.constant(csizes_scale, dtype = tf.float32)
#pen_bin = tf.reduce_sum((.25 - (.5 - x)**2)**2)
pen_bin = tf.reduce_sum(tf.math.minimum(tf.math.abs(x), tf.math.abs(x - 1)))
pen_count = tf.reduce_sum((tf.reduce_sum(x**2, axis=0) - nsub)**2)
#pen_count = tf.reduce_sum(tf.math.maximum(0.0, tf.reduce_sum(x, axis=0) - nsub))
loss = tf.reduce_sum((tf.matmul(C, x) - cs)**2)
objf = lam_loss * loss + lam_bin * pen_bin + lam_count * pen_count

In [114]:
opt = tf.train.AdamOptimizer(learning_rate = 0.001, beta1=0.9, beta2=0.999, epsilon=1e-8)
grads = opt.compute_gradients(objf)
apply_gradient_op = opt.apply_gradients(grads)

In [116]:
nits = 20000
with tf.Session() as sess:
    tf.global_variables_initializer().run(session=sess)
    for it_no in range(nits):
        lam_bin_v = 100 * np.power(30 + it_no, 0.7)
        lam_count_v = 100 * np.power(1 + it_no, 0.5)
        lam_loss_v = 0 # 200/(200 + it_no)
        l, o, pb, pc, _ = sess.run([loss, objf, pen_bin, pen_count, apply_gradient_op], 
                           feed_dict={lam_bin:lam_bin_v, lam_count:lam_count_v, lam_loss:lam_loss_v})
        if it_no % 1000 == 0:
            print({'objf':o, 'loss':l, 'pen_bin': pb, 'pen_count': pc})
    x_found = sess.run(x)

{'objf': 946548.0, 'loss': 15726.863, 'pen_bin': 841.78784, 'pen_count': 362.41788}
{'objf': 4255623.0, 'loss': 7354.004, 'pen_bin': 321.55045, 'pen_count': 38.848415}
{'objf': 6815443.5, 'loss': 7493.035, 'pen_bin': 318.9824, 'pen_count': 49.962105}
{'objf': 8997110.0, 'loss': 7585.802, 'pen_bin': 317.27878, 'pen_count': 58.137592}
{'objf': 10962679.0, 'loss': 7657.548, 'pen_bin': 315.9734, 'pen_count': 64.81346}
{'objf': 12781298.0, 'loss': 7716.0723, 'pen_bin': 314.90958, 'pen_count': 70.547}
{'objf': 14490372.0, 'loss': 7765.498, 'pen_bin': 314.0076, 'pen_count': 75.60095}
{'objf': 16113143.0, 'loss': 7810.037, 'pen_bin': 313.20346, 'pen_count': 80.24637}
{'objf': 17665622.0, 'loss': 7849.623, 'pen_bin': 312.49542, 'pen_count': 84.47178}
{'objf': 19159066.0, 'loss': 7884.205, 'pen_bin': 311.8719, 'pen_count': 88.29732}
{'objf': 20601646.0, 'loss': 7916.8335, 'pen_bin': 311.29227, 'pen_count': 91.91574}
{'objf': 21999814.0, 'loss': 7946.981, 'pen_bin': 310.75476, 'pen_count': 95.317

In [109]:
x_round = (x_found > 0.5) + 0.0

In [117]:
np.sum(x_found, 0)

array([42.585194, 43.36702 , 44.12913 , 42.978504, 43.368404, 47.09666 ,
       43.36767 , 43.754204, 42.58819 , 43.75285 , 41.788525, 44.138966,
       42.98011 , 42.979416, 45.63634 , 44.138416, 43.367188, 44.135437,
       44.892277, 44.892708], dtype=float32)

In [111]:
np.sum(x_round, 0)

array([61., 61., 64., 60., 63., 62., 57., 61., 54., 57., 63., 50., 57.,
       60., 49., 60., 61., 55., 56., 59.])